In [ ]:
#| default_exp release

# release

> Auto-generated tagged releases and release notes from GitHub issues

## Overview

`nbdev.release` provides 3 commands that you can run from your shell:

- `nbdev_changelog`: creates a CHANGELOG.md file from closed and labeled GitHub issues
- `nbdev_release_git`: tags and creates a release in GitHub for the current version
- `nbdev_release_gh`: calls `nbdev_changelog`, lets you edit the result, then pushes to git and calls `nbdev_release_git`

Here's a brief demonstration of how to use the tools in `nbdev.release`. This demo first creates an issue using the [`gh`](https://cli.github.com/) command line tool, and then closes it using `git`; you can also use GitHub's web interface for both of these tasks. (Note that this functionality used to be in a project called `fastrelease`, so in the video the command line tools have different names, starting with `fastrelease_` instead of `nbdev_`).

<a href="images/intro.svg"><img src="images/release.svg" width="900px" /></a>

### Setup

First, create a `settings.ini` file with the following contents (replacing the values as described below):

```
[DEFAULT]
lib_name = fastrelease
user = fastai
version = 0.0.1
```

Set `lib_name` to the name of GitHub repo, `user` to the owner of that repo, and `version` to the version number of your library. (Note that if you use [nbdev](https://nbdev.fast.ai) then you'll already have this information, so you don't need to do anything further to set it up.)

You'll need to get a GitHub [personal access token](https://docs.github.com/en/github/authenticating-to-github/creating-a-personal-access-token) if you haven't already. To do so, [click here](https://github.com/settings/tokens/new) and enter "fastrelease" in the "Note" section, and click the `repo` checkbox.

Then click "Generate Token" at the bottom of the screen, and copy the token (the long string of letters and numbers shown). You can easily do that by clicking the little clipboard icon next to the token.

<img alt="Copying your token" width="743" caption="Copying your token" src="images/token.png">

Paste that token into a file called `token` into the root of your repo. You can run the following in your terminal (`cd` to the root of your repo first) to create that file:

    echo XXX > token

Replace *XXX* above with the token you copied. Also, ensure that this file isn't added to git, by running this in your terminal:

    echo token >> .gitignore

### Creating release notes

Now you're ready to create your release notes. These are created in a file called `CHANGELOG.md`. Here's an example of what it creates: [nbdev CHANGELOG](https://github.com/fastai/nbdev/blob/master/CHANGELOG.md).

All issues with the label **bug**, **enhancement**, or **breaking** that have been closed in your repo since your last release will be added to the top of this file. If you haven't made any releases before, then all issues with those labels will be included.

Therefore, before you create or update `CHANGELOG.md`, go to your GitHub issues page, remove `is:open` from the filter, and label any issues you want included with one of the labels above. When you've done that, you can create or update your release notes by running in your terminal:

    nbdev_changelog

The titles and bodies of each issue will be added. Open `CHANGELOG.md` in your editor and make any edits that you want, and then commit the file to your repo (remember to `git add` it!)

### Tagging a release

You should now tag a release. This will create a tag in GitHub with your current version number in `settings.ini`, and will then make it into a release, using your latest release notes as the description of the release:

    nbdev_release_git

After you run this, be sure to increment your version number in `settings.ini`. You can either edit it manually, or if you use nbdev it can be done for you by running:

    nbdev_bump_version

### Doing both (creating release notes, and tagging a release)

To complete both of the steps above, run:

```
nbdev_release_gh
```

See the screencast above for a demonstration of this.

## Python API

In [ ]:
#| export
from fastcore.all import *
from ghapi.core import *

from datetime import datetime
from configparser import ConfigParser
import shutil,subprocess

In [ ]:
#| hide
from nbdev.showdoc import show_doc

In [ ]:
#| export
GH_HOST = "https://api.github.com"

In [ ]:
#| export
def _find_config(cfg_name="settings.ini"):
    cfg_path = Path().absolute()
    while cfg_path != cfg_path.parent and not (cfg_path/cfg_name).exists(): cfg_path = cfg_path.parent
    return Config(cfg_path, cfg_name)

In [ ]:
#| export
def _issue_txt(issue):
    res = '- {} ([#{}]({}))'.format(issue.title.strip(), issue.number, issue.html_url)
    if hasattr(issue, 'pull_request'): res += ', thanks to [@{}]({})'.format(issue.user.login, issue.user.html_url)
    res += '\n'
    if not issue.body: return res
    return res + f"  - {issue.body.strip()}\n"

def _issues_txt(iss, label):
    if not iss: return ''
    res = f"### {label}\n\n"
    return res + '\n'.join(map(_issue_txt, iss))

def _load_json(cfg, k):
    try: return json.loads(cfg[k])
    except json.JSONDecodeError as e: raise Exception(f"Key: `{k}` in .ini file is not a valid JSON string: {e}")

## Release -

In [ ]:
#| export
class Release:
    def __init__(self, owner=None, repo=None, token=None, **groups):
        "Create CHANGELOG.md from GitHub issues"
        self.cfg = _find_config()
        self.changefile = self.cfg.config_path/'CHANGELOG.md'
        if not groups:
            default_groups=dict(breaking="Breaking Changes", enhancement="New Features", bug="Bugs Squashed")
            groups=_load_json(self.cfg, 'label_groups') if 'label_groups' in self.cfg else default_groups
        os.chdir(self.cfg.config_path)
        owner,repo = owner or self.cfg.user, repo or self.cfg.lib_name
        token = ifnone(token, os.getenv('nbdev_TOKEN',None))
        if not token and Path('token').exists(): token = Path('token').read_text().strip()
        if not token: raise Exception('Failed to find token')
        self.gh = GhApi(owner, repo, token)
        self.groups = groups

    def _issues(self, label):
        return self.gh.issues.list_for_repo(state='closed', sort='created', filter='all', since=self.commit_date, labels=label)
    def _issue_groups(self): return parallel(self._issues, self.groups.keys(), progress=False)

To create a markdown changelog, first create a `Release` object, optionally passing a mapping from GitHub labels to markdown titles. Put your github token in a file named `token` at the root of your repo.  `Release` attempts to fetch values for arguments from the following locations if not supplied:

- **owner:** fetched from the field `user` in `settings.ini`.  This is the owner name of the repository on GitHub. For example for the repo `fastai/fastcore` the owner would be `fastai`.
- **repo:** fetched from the field `lib_name` in `settings.ini`.  This is the name of the repository on GitHub.  For example for the repo `fastai/fastcore` the owner would be `fastcore`.
- **token:** fetched from a file named `token` at the root of your repo.  Creating a token is discussed in [the setup](https://fastrelease.fast.ai/#Set-up) section.
- **groups:** (optional) fetched from the field `label_groups` in `settings.ini`, which is a JSON string.  This is a mapping from label names to titles in your release notes. If not specified, this defaults to:

```python
{"breaking": "Breaking Changes", "enhancement":"New Features", "bug":"Bugs Squashed"}
```

In [ ]:
#|export
@patch
def changelog(self:Release,
              debug=False): ## Just print the latest changes, instead of updating file
    "Create the CHANGELOG.md file, or return the proposed text if `debug` is `True`"
    if not self.changefile.exists(): self.changefile.write_text("# Release notes\n\n<!-- do not remove -->\n")
    marker = '<!-- do not remove -->\n'
    try: self.commit_date = self.gh.repos.get_latest_release().published_at
    except HTTP404NotFoundError: self.commit_date = '2000-01-01T00:00:004Z'
    res = f"\n## {self.cfg.version}\n"
    issues = self._issue_groups()
    res += '\n'.join(_issues_txt(*o) for o in zip(issues, self.groups.values()))
    if debug: return res
    res = self.changefile.read_text().replace(marker, marker+res+"\n")
    shutil.copy(self.changefile, self.changefile.with_suffix(".bak"))
    self.changefile.write_text(res)
    run(f'git add {self.changefile}')

In [ ]:
#| eval: false
rel = Release()
print(rel.changelog(debug=True))


## 2.1.1


### Bugs Squashed

- fix `nbdev_test` with no `--fname` in non-nbdev repos ([#730](https://github.com/fastai/nbdev/pull/730)), thanks to [@seeM](https://github.com/seeM)

- Auto-generated showdoc headers not in ToC ([#703](https://github.com/fastai/nbdev/issues/703))
  - Should have been fixed in Quarto here:

https://github.com/quarto-dev/quarto-cli/commit/dbb9de99a1ad959d1d6e064654002a8247138289

Check whether `#| output: asis` fixes it.



In [ ]:
#|export
@patch
def release(self:Release):
    "Tag and create a release in GitHub for the current version"
    ver = self.cfg.version
    notes = self.latest_notes()
    self.gh.create_release(ver, branch=self.cfg.branch, body=notes)
    return ver

This uses the version information from your `settings.ini`.

In [ ]:
#|export
@patch
def latest_notes(self:Release):
    "Latest CHANGELOG entry"
    if not self.changefile.exists(): return ''
    its = re.split(r'^## ', self.changefile.read_text(), flags=re.MULTILINE)
    if not len(its)>0: return ''
    return '\n'.join(its[1].splitlines()[1:]).strip()

All relevant pull requests and issues are fetched from the GitHub API, and are categorized according to a user-supplied mapping from labels to markdown headings.

## CLI functions

In [ ]:
#| export
@call_parse
def changelog(
    debug:store_true=False,  # Print info to be added to CHANGELOG, instead of updating file
    repo:str=None,  # repo to use instead of `lib_name` from `settings.ini`
):
    "Create a CHANGELOG.md file from closed and labeled GitHub issues"
    res = Release(repo=repo).changelog(debug=debug)
    if debug: print(res)

In [ ]:
#| export
@call_parse
def release_git(
    token:str=None  # Optional GitHub token (otherwise `token` file is used)
):
    "Tag and create a release in GitHub for the current version"
    ver = Release(token=token).release()
    print(f"Released {ver}")

In [ ]:
#| export
@call_parse
def release_gh(
    token:str=None  # Optional GitHub token (otherwise `token` file is used)
):
    "Calls `nbdev_changelog`, lets you edit the result, then pushes to git and calls `nbdev_release_git`"
    cfg = _find_config()
    Release().changelog()
    subprocess.run([os.environ.get('EDITOR','nano'), cfg.config_path/'CHANGELOG.md'])
    if not input("Make release now? (y/n) ").lower().startswith('y'): sys.exit(1)
    run('git commit -am release')
    run('git push')
    ver = Release(token=token).release()
    print(f"Released {ver}")

## Export -

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()